In [1]:
import sympy
import numpy as np
sympy.init_printing(use_latex=True)

### Example provided in '011- EKF tutorial' in rlabbe's repo

In [2]:
x, x_vel, y = sympy.symbols('x, x_vel y')
H = sympy.Matrix([sympy.sqrt(x**2 + y**2)])
state = sympy.Matrix([x, x_vel, y])
H.jacobian(state)

⎡     x                y      ⎤
⎢────────────  0  ────────────⎥
⎢   _________        _________⎥
⎢  ╱  2    2        ╱  2    2 ⎥
⎣╲╱  x  + y       ╲╱  x  + y  ⎦

### Motion model for Differential robot referred in Andrew Kramer's blog

In [3]:
#state variables
x, y, theta = sympy.symbols('x, y, theta', real=True)

#input variables
v_t, omega_t = sympy.symbols('v_t, omega_t', real=True)

# measurement variables
l_x, l_y, l_id = sympy.symbols('l_x, l_y, l_id', real=True)

#common variables
delta_t      = sympy.symbols('delta_t', real=True)

In [10]:
subs_dict = {x:0.0, y:0.0, theta:0.0, v_t:0.0, omega_t:0.0, delta_t :0.0}

In [11]:
motion_model = sympy.Matrix([
                            [x + v_t * delta_t * sympy.cos(theta + (omega_t * delta_t / 2.0))],
                            [y + v_t * delta_t * sympy.sin(theta + (omega_t * delta_t / 2.0))],
                            [theta + omega_t * delta_t]
                        ])

state_vector = sympy.Matrix([x, y, theta])
input_vector = sympy.Matrix([v_t, omega_t])

In [12]:
mot_output = motion_model.evalf(subs = subs_dict)

In [15]:
u = np.array(mot_output).astype(float)
print(u)

[[1.18182126e-125]
 [1.18182126e-125]
 [1.18182126e-125]]


### Computing jacobians for motion model wrt different inputs

In [16]:
motion_model.jacobian(state_vector)

⎡1  0  -δₜ⋅vₜ⋅sin(0.5⋅δₜ⋅ωₜ + θ)⎤
⎢                               ⎥
⎢0  1  δₜ⋅vₜ⋅cos(0.5⋅δₜ⋅ωₜ + θ) ⎥
⎢                               ⎥
⎣0  0              1            ⎦

In [17]:
motion_model.jacobian(input_vector)

⎡                              2                      ⎤
⎢δₜ⋅cos(0.5⋅δₜ⋅ωₜ + θ)  -0.5⋅δₜ ⋅vₜ⋅sin(0.5⋅δₜ⋅ωₜ + θ)⎥
⎢                                                     ⎥
⎢                             2                       ⎥
⎢δₜ⋅sin(0.5⋅δₜ⋅ωₜ + θ)  0.5⋅δₜ ⋅vₜ⋅cos(0.5⋅δₜ⋅ωₜ + θ) ⎥
⎢                                                     ⎥
⎣          0                          δₜ              ⎦

### Evaluating jacobian at some point and converting to np array

In [20]:
F = motion_model.jacobian(state_vector)
F.evalf(subs = subs_dict)

⎡1.0   0    0 ⎤
⎢             ⎥
⎢ 0   1.0   0 ⎥
⎢             ⎥
⎣ 0    0   1.0⎦

In [21]:
a = F.evalf(subs = subs_dict)

In [22]:
type(a)

sympy.matrices.dense.MutableDenseMatrix

In [23]:
u = np.array(a).astype(float)

In [24]:
u

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

### Measurement model

In [25]:
measurement_model = sympy.Matrix([
                                [sympy.sqrt((l_x-x)**2 + (l_y-y)**2)],
                                [sympy.atan2(l_y-y, l_x-x) - theta],
                                [l_id]
                                ])

In [26]:
measurement_model.jacobian(state_vector)

⎡          -lₓ + x                      -l_y + y             ⎤
⎢───────────────────────────  ───────────────────────────  0 ⎥
⎢   ________________________     ________________________    ⎥
⎢  ╱         2            2     ╱         2            2     ⎥
⎢╲╱  (lₓ - x)  + (l_y - y)    ╲╱  (lₓ - x)  + (l_y - y)      ⎥
⎢                                                            ⎥
⎢       -(-l_y + y)                   -(lₓ - x)              ⎥
⎢  ──────────────────────       ──────────────────────     -1⎥
⎢          2            2               2            2       ⎥
⎢  (lₓ - x)  + (l_y - y)        (lₓ - x)  + (l_y - y)        ⎥
⎢                                                            ⎥
⎣             0                            0               0 ⎦

### Conclusion

https://docs.sympy.org/latest/modules/numeric-computation.html

As described in the link above, the method above cannot be used to evaluate jacobians for a real-time filter, as
it consumes a lot of time

Purpose
-------
1. Compute jacobians for those hard to solve by hand
2. Check correctness of jacobians already computed
3. Simplify expressions for easy understanding